In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,accuracy_score,mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import joblib
import warnings
warnings.filterwarnings('ignore')

In [9]:
df = pd.read_csv("prep_data.csv")
df = df.drop(columns=['followup1_date', 'followup2_date', 'surgery_date','surgery_type'])

In [10]:
X = df.drop(columns=[
    'followup1_vision_left', 'followup1_vision_right',
    'followup1_refraction_sph_left', 'followup1_refraction_cyl_left',
    'followup1_refraction_sph_right', 'followup1_refraction_cyl_right',
    'followup1_keratometry_left', 'followup1_keratometry_right',
    'followup1_complication', 'followup2_complication'
])

In [11]:
y_reg = df[[
    'followup1_vision_left', 'followup1_vision_right',
    'followup1_refraction_sph_left', 'followup1_refraction_cyl_left',
    'followup1_refraction_sph_right', 'followup1_refraction_cyl_right',
    'followup1_keratometry_left', 'followup1_keratometry_right'
]]

In [12]:
y_class1 = df['followup1_complication']
y_class2 = df['followup2_complication']

In [13]:
X_train, X_test, y_reg_train, y_reg_test, y_class1_train, y_class1_test, y_class2_train, y_class2_test = train_test_split(
    X, y_reg, y_class1, y_class2, test_size=0.2, random_state=42
)

In [14]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
joblib.dump(scaler, "model_training_scaler.pkl")

['model_training_scaler.pkl']

In [15]:
input_reg = Input(shape=(X_train.shape[1],))
x = Dense(128, activation='relu')(input_reg)
x = Dropout(0.4)(x)
x = Dense(64, activation='relu')(x)
output_reg = Dense(8, name='regression')(x)

In [16]:
reg_model = Model(inputs=input_reg, outputs=output_reg)
reg_model.compile(optimizer=Adam(0.001), loss='mse', metrics=['mae'])
reg_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 36)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           4,736 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ regression (Dense)                   │ (None, 8)                   │             520 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,512 (52.78 KB)

 Trainable params: 13,512 (52.78 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
reg_history = reg_model.fit(
    X_train,
    y_reg_train,
    validation_split=0.1,
    epochs=45,
    batch_size=32,
    verbose=1
)

Epoch 1/45
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - loss: 484.1276 - mae: 11.6090 - val_loss: 489.7827 - val_mae: 11.6362
Epoch 2/45
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 483.7498 - mae: 11.6004 - val_loss: 489.4035 - val_mae: 11.6279
Epoch 3/45
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 483.3723 - mae: 11.5919 - val_loss: 489.0247 - val_mae: 11.6198
Epoch 4/45
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 482.9951 - mae: 11.5835 - val_loss: 488.6461 - val_mae: 11.6115
Epoch 5/45
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 482.6185 - mae: 11.5752 - val_loss: 488.2682 - val_mae: 11.6035
Epoch 6/45
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 482.2423 - mae: 11.5669 - val_loss: 487.8905 - val_mae: 11.5954
Epoch 7/45
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 481.8666 - mae: 11.5588 - val_loss: 487.5131 - val_mae: 11.5875
Epoch 8/45
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 481.4911 - mae: 11.5508 - val_loss: 487.1359 - val_mae: 11.5799
Epoch 9/45
17/17

In [20]:
reg_preds = reg_model.predict(X_test)
print("\nMean Absolute Error on Regression Test Set:", mean_absolute_error(y_reg_test, reg_preds))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

Mean Absolute Error on Regression Test Set: 11.369517805324389


In [21]:
input_class = Input(shape=(X_train.shape[1],))
x = Dense(128, activation='relu')(input_class)
x = Dropout(0.45)(x)
x = Dense(64, activation='relu')(x)

In [22]:
output_class1 = Dense(len(np.unique(y_class1)), activation='softmax', name='class1')(x)
output_class2 = Dense(len(np.unique(y_class2)), activation='softmax', name='class2')(x)

In [23]:
class_model = Model(inputs=input_class, outputs=[output_class1, output_class2])
class_model.compile(
    optimizer='adam',
    loss={
        'class1': 'sparse_categorical_crossentropy',
        'class2': 'sparse_categorical_crossentropy'
    },
    metrics={
        'class1': 'accuracy',
        'class2': 'accuracy'
    }
)

class_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)    │ (None, 36)                │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 128)               │           4,736 │ input_layer_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 128)               │               0 │ dense_2[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 64)                │           8,256 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ class1 (Dense)                │ (None, 6)                 │             390 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ class2 (Dense)                │ (None, 6)                 │             390 │ dense_3[0][0]              │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 13,772 (53.80 KB)

 Trainable params: 13,772 (53.80 KB)

 Non-trainable params: 0 (0.00 B)

In [30]:
class_history = class_model.fit(
    X_train,
    [y_class1_train, y_class2_train],
    validation_split=0.1,
    epochs=35,
    batch_size=32,
    verbose=0
)

In [31]:
class_preds = class_model.predict(X_test)
pred_class1 = np.argmax(class_preds[0], axis=1)
pred_class2 = np.argmax(class_preds[1], axis=1)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [32]:
print("\nAccuracy for Followup1 Complications:")
print(accuracy_score(y_class1_test, pred_class1)*100)


Accuracy for Followup1 Complications:
69.33333333333334


In [33]:
print("\nClassification Report for Followup2 Complications:")
print(accuracy_score(y_class2_test, pred_class2)*100)


Classification Report for Followup2 Complications:
78.0
